In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits import mplot3d

import math
from sklearn.model_selection import train_test_split
import pickle

In [6]:
# load features data ; change file path accordingly
fpath = "./data/features/sorted.pkl"
data = pickle.load(open(fpath,'rb'))
labels = pd.read_csv("./data/labels/cleaned_labels.csv", index_col=0)

In [9]:
data.groupby('bookingID').count().head(20)

,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed
bookingID,,,,,,,,,,
0,1004,1004,1004,1004,1004,1004,1004,1004,1004,1004
1,851,851,851,851,851,851,851,851,851,851
10,375,375,375,375,375,375,375,375,375,375
1005022347266,840,840,840,840,840,840,840,840,840,840
1005022347267,740,740,740,740,740,740,740,740,740,740
1005022347268,428,428,428,428,428,428,428,428,428,428
1005022347269,1595,1595,1595,1595,1595,1595,1595,1595,1595,1595
1005022347271,420,420,420,420,420,420,420,420,420,420
1005022347275,630,630,630,630,630,630,630,630,630,630


In [69]:
# Using 10% of the data first
df_mini = data.head(int(len(data)*0.1)).copy()
df_mini = df_mini.reset_index()

# ensure that the last value we chop off is a complete bookingID's 
lastval = df_mini[(df_mini.bookingID==df_mini.iloc[-1,1]) & (df_mini.second == 0.00)].index[0]
df_mini = df_mini.iloc[:lastval,:]
df_mini.drop('index',inplace=True, axis=1)
len(df_mini)

1611105

In [64]:
df_mini.tail(20)
df_mini.head(20)

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed
0,0,12.0,143.298294,0.818112,-9.941461,-2.014999,-0.016245,-0.094040,0.070732,0.0,3.442991
1,0,8.0,143.298294,0.546405,-9.835590,-2.038925,-0.047092,-0.078874,0.043187,1.0,0.228454
2,0,8.0,143.298294,-1.706207,-9.270792,-1.209448,-0.028965,-0.032652,0.015390,2.0,0.228454
3,0,8.0,143.298294,-1.416705,-9.548032,-1.860977,-0.022413,0.005049,-0.025753,3.0,0.228454
4,0,8.0,143.298294,-0.598145,-9.853534,-1.378574,-0.014297,-0.046206,0.021902,4.0,0.228454
5,0,8.0,143.298294,-0.608313,-9.539658,-1.794583,-0.007538,-0.023838,0.018068,5.0,0.228454
6,0,8.0,143.298294,-0.867758,-9.698615,-1.615439,0.022728,-0.012178,0.005982,6.0,0.228454
7,0,8.0,143.298294,-1.050790,-9.745270,-1.411771,0.027603,0.001841,0.000904,7.0,0.228454
8,0,8.0,143.298294,-0.721213,-9.960004,-1.202271,0.001864,-0.007702,0.014018,8.0,0.228454
9,0,8.0,143.298294,-0.346924,-9.532629,-1.204663,0.014962,-0.050033,0.025118,9.0,0.228454


In [130]:
df_totrain = data.groupby('bookingID').agg(['mean','std','min','max'])

In [49]:
# labels.drop('Unnamed: 0',inplace=True,axis=1) # Needed if index column is passed to csv 
labels.sort_values('bookingID',inplace=True)

In [85]:
startoftrip = data[data.second == 0]
startoftrip.head()

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed
1153972,0,12.0,143.298294,0.818112,-9.941461,-2.014999,-0.016245,-0.094040,0.070732,0.0,3.442991
212430,1,4.0,252.000000,-1.394632,10.174141,-2.503770,0.016127,0.212284,-0.039776,0.0,4.756427
1312695,10,3.0,0.000000,0.928955,9.296692,-1.161194,-0.013931,-0.007431,0.006516,0.0,0.000000
75086,1005022347266,3.0,95.000000,-0.906225,9.547540,-1.969930,0.009390,0.004643,-0.003595,0.0,0.000000
1049487,1005022347267,3.9,181.000000,0.683411,10.746094,2.123398,0.007446,0.011063,-0.017456,0.0,22.760000


In [104]:
labels_id = labels.bookingID 
data_id = data.bookingID.unique()
len(labels_id), len(data_id)

In [114]:
data.groupby('bookingID').second.count().head(20)

bookingID
0                1004
1                 851
10                375
1005022347266     840
1005022347267     740
1005022347268     428
1005022347269    1595
1005022347271     420
1005022347275     630
1005022347277    1050
1005022347279     630
1005022347280     225
1005022347282     839
1005022347284    1045
1005022347285     327
1005022347289    1485
1005022347291     522
1005022347293    1394
1005022347294     443
1005022347296    1015
Name: second, dtype: int64

In [119]:
# Cleaning: records with sudden jump in recorded seconds 
data[data.second > 10000].groupby('bookingID').count()

,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed
bookingID,,,,,,,,,,
1108101562401,3,3,3,3,3,3,3,3,3,3
1434519076976,2,2,2,2,2,2,2,2,2,2
1460288880770,32,32,32,32,32,32,32,32,32,32
1503238553722,1,1,1,1,1,1,1,1,1,1
858993459333,44,44,44,44,44,44,44,44,44,44


In [129]:
# individual to analyze; change bookingID as needed
indiv_data = data[data.bookingID == '858993459333']
print(len(indiv_data))
indiv_data.sort_values('second').head(20)

504


,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed
435937,858993459333,12.000,353.0,-0.632069,9.643845,3.122039,-0.051007,0.028100,0.082161,0.0,5.358241
1311796,858993459333,12.000,353.0,-0.670377,9.567230,3.677494,-0.037263,0.045509,0.014966,1.0,5.141538
162552,858993459333,10.077,354.0,-0.756568,9.643845,3.198653,-0.056505,0.022602,0.028711,2.0,2.623132
181512,858993459333,10.011,354.0,-0.306458,9.653421,1.628057,-0.048564,0.032070,0.028100,3.0,0.000000
487156,858993459333,9.963,354.0,-0.191536,9.816227,1.292869,-0.030238,0.025351,0.025045,4.0,0.000000
950959,858993459333,9.930,354.0,-0.354342,9.883265,0.258574,-0.050396,0.045815,0.019853,5.0,0.000000
1328550,858993459333,9.905,354.0,2.078167,11.013328,5.631162,-0.079107,-0.039095,-0.001222,6.0,0.000000
877628,858993459333,10.240,358.0,0.363919,9.950302,0.124498,-0.106291,-0.214414,0.046120,7.0,3.194665
516470,858993459333,10.226,14.0,0.737414,10.180145,1.628057,0.028100,-0.257785,-0.017410,8.0,3.194665
1532722,858993459333,10.193,33.0,1.829170,9.825804,1.331176,-0.028100,-0.432493,0.013744,9.0,2.101302
